# Public, Protected, Private

## Example

In [4]:
class User:
    def __init__(self, name, birth_day, password):
        self.name = name  # Public
        self._birthday = birth_day  # Protected
        self.__password = password  # Private

    def check_birth_day(self, input_birth_day):
        return input_birth_day == self._birthday  # Accessing protected member

    def check_password(self, input_password):
        return input_password == self.__password  # Accessing private member

    def get_birth_day(self):  # Example of a getter method
        return self._birthday

    def set_birth_day(self, new_birth_day): # Example of a setter method
        self._birthday = new_birth_day

user = User("Alice", "1990-01-01", "secure123")
print(user.name)  # ✅ Can access (Public)

# Accessing protected member directly (generally discouraged but possible)
print(user._birthday)  # ⚠️ Works, but considered bad practice outside the class

# Recommended way to access or modify protected members:
print(user.get_birth_day()) # Using a getter method
user.set_birth_day("1995-05-05") # Using a setter method
print(user.get_birth_day())

# print(user.__password)  # ❌ AttributeError: 'User' object has no attribute '__password'

# But Python allows indirect access (name-mangling) - Avoid this!
# print(user._User__password)  # ⚠️ Works, but bad practice -  Directly accessing mangled name

# Example demonstrating why protected variables are useful:
class AdminUser(User):
    def __init__(self, name, birth_day, password, access_level):
        super().__init__(name, birth_day, password)
        self._access_level = access_level # Protected variable specific to AdminUser

    def get_access_level(self):
        return self._access_level

admin = AdminUser("Bob", "1985-11-15", "adminpass", "full")
print(admin.name) # Public
print(admin._birthday) # Accessible, but bad practice
print(admin.get_access_level()) # Recommended access for protected _access_level

Alice
1990-01-01
1990-01-01
1995-05-05
Bob
1985-11-15
full


## Bad vs Good example

the following code uses bad practices

In [5]:
import datetime

# Bad Practice (Public attribute)

class UserBad:
    def __init__(self, name, birthday_str):
        self.name = name
        self.birthday = birthday_str  # Public attribute

    def display_birthday(self):
        print(f"Birthday: {self.birthday}")

user_bad = UserBad("Alice", "1990-01-01")
user_bad.display_birthday()  # Output: Birthday: 1990-01-01

# ... later in your code ...
user_bad.birthday = "Invalid Date"  # Directly modifying the public attribute
user_bad.display_birthday()  # Output: Birthday: Invalid Date (Oops!)

# ... even later, you decide to change the birthday to a datetime object ...
user_bad.birthday = datetime.datetime.strptime("1990-01-01", "%Y-%m-%d").date()
user_bad.display_birthday() # This will now cause an error because the display method expects a string.
print(user_bad.birthday.year) # This will work, but other parts of the code may be expecting a string.

# Good Practice (Protected attribute with getters/setters)

class UserGood:
    def __init__(self, name, birthday_str):
        self.name = name
        self._birthday = datetime.datetime.strptime(birthday_str, "%Y-%m-%d").date() # Store as datetime.date object from the start

    def get_birthday(self):
        return self._birthday.strftime("%Y-%m-%d") # Return as string when needed

    def set_birthday(self, birthday_str):
        try:
            self._birthday = datetime.datetime.strptime(birthday_str, "%Y-%m-%d").date()
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    def display_birthday(self):
        print(f"Birthday: {self.get_birthday()}")  # Use the getter

user_good = UserGood("Bob", "1985-05-10")
user_good.display_birthday()  # Output: Birthday: 1985-05-10

# ... later in your code ...
user_good.set_birthday("2000-01-01")  # Using the setter
user_good.display_birthday()  # Output: Birthday: 2000-01-01

user_good.set_birthday("Invalid Date") # Output: Invalid date format. Please use YYYY-MM-DD.
user_good.display_birthday() # Output: Birthday: 2000-01-01 (No change)

print(user_good.get_birthday()) # Accessing the birthday as a string.
print(user_good._birthday.year) # Accessing the birthday as a date object.

# ... even later, you decide you want to change how the birthday is stored. You can change it to a timestamp, for instance.
# No changes are needed in any other part of the code, as long as the getter and setter are updated.

Birthday: 1990-01-01
Birthday: Invalid Date
Birthday: 1990-01-01
1990
Birthday: 1985-05-10
Birthday: 2000-01-01
Invalid date format. Please use YYYY-MM-DD.
Birthday: 2000-01-01
2000-01-01
2000


### I have a question

**im still confused, doesnt a setter change the attribute anyways?**


You're absolutely right to be confused! It's a subtle but crucial point. Yes, a setter *does* change the attribute.  But the way it changes it, and the control it provides, is what makes it different and *better* than directly assigning to a public attribute.

Let's illustrate with an analogy:

Imagine you have a valuable object (your data, like `_birthday`).

*   **Public Attribute (Direct Access):**  It's like leaving the object out in the open. Anyone can come along and change it, even accidentally or inappropriately (like changing its type or setting it to an invalid value). You have no record of who changed it or when.
*   **Setter Method:** It's like having a secure box for the object.  To change the object, you have to go through a specific process:
    1.  You hand the new version of the object (the new birthday value) to a designated person (the setter method).
    2.  The designated person (the setter) can then:
        *   Check if the new object is valid (e.g., correct date format).
        *   Record who made the change and when (you could add logging).
        *   Perform any necessary conversions or adjustments before placing the new object in the box (e.g., convert the string to a `datetime` object).
    3.  Finally, they replace the old object with the new one.

**Key Differences and Why Setters Are Better:**

1.  **Validation:** The setter can *validate* the new value *before* it's assigned to the attribute.  This prevents errors and ensures data integrity.  Direct assignment to a public attribute bypasses this validation.

2.  **Abstraction:** The setter *hides the internal details* of how the data is stored.  You can change how `_birthday` is stored (e.g., from a string to a `datetime` object) *without* affecting the code that uses the setter.  With direct access, you'd have to change every piece of code that touches the attribute.

3.  **Control:** The setter gives you *control* over when and how the attribute is changed.  You can add logging, security checks, or other logic to the setter.  Direct assignment provides no such control.

4.  **Maintainability:**  If you need to change how the attribute is handled, you only need to change the setter (and getter, if necessary).  With direct access, you'd have to find and modify *every instance* where the attribute is used.

# End